In [1]:
import cdsapi

dataset = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": 
        "2m_temperature"
    ,
    "year": [
        "1980", "1981", "1982",
        "1983", "1984", "1985",
        "1986", "1987", "1988",
        "1989", "1990", "1991",
        "1992", "1993", "1994",
        "1995", "1996", "1997",
        "1998", "1999", "2000",
        "2001", "2002", "2003",
        "2004", "2005", "2006",
        "2007", "2008", "2009",
        "2010", "2011", "2012",
        "2013", "2014", "2015",
        "2016", "2017"
    ],
    "month": [
        "05", "06", "07",
        "08"
    ],
    "time": ["00:00"],
    "data_format": "grib",
    "download_format": "unarchived",
    "area": [38.5, 59.5, 22.5, 78.5]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


2025-01-25 10:46:46,216 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2025-01-25 10:46:46,226 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-01-25 10:46:46,226 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2025-01-25 10:46:46,230 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using t

'3820e8bd74f5afc20795ce4218d3dfa6.grib'

In [2]:
import xarray as xr
ds1= xr.open_dataset(r'D:\FYP\active\3820e8bd74f5afc20795ce4218d3dfa6.grib')


In [3]:
ds1


<xarray.Dataset> Size: 3MB
Dimensions:     (time: 152, latitude: 65, longitude: 77)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 1kB 1980-05-01 1980-06-01 ... 2017-08-01
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 520B 38.5 38.25 38.0 ... 23.0 22.75 22.5
  * longitude   (longitude) float64 616B 59.5 59.75 60.0 ... 78.0 78.25 78.5
    valid_time  (time) datetime64[ns] 1kB ...
Data variables:
    t2m         (time, latitude, longitude) float32 3MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-25T10:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
# Calculate mean temperature across time dimension
mean_temp = ds1.t2m.mean(dim='time')

# Convert from Kelvin to Celsius
mean_temp_celsius = mean_temp - 273.15

# Create point-based temperature data
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Create points from temperature grid
lons, lats = np.meshgrid(mean_temp.longitude, mean_temp.latitude)
temp_df = pd.DataFrame({
    'longitude': lons.flatten(),
    'latitude': lats.flatten(),
    'temperature': mean_temp_celsius.values.flatten()
})

# Convert to GeoDataFrame
temp_gdf = gpd.GeoDataFrame(
    temp_df, 
    geometry=[Point(xy) for xy in zip(temp_df.longitude, temp_df.latitude)],
    crs="EPSG:4326"
)

# Read districts shapefile
districts = gpd.read_file('D:/FYP/active/shape/District_Boundary.shp')

# Spatial join and calculate district means
district_temps = gpd.sjoin(districts, temp_gdf, how='left', predicate='contains')
result = district_temps.groupby('DISTRICT')['temperature'].mean().round(2).reset_index()

# Sort by temperature and display nicely
result = result.sort_values('temperature', ascending=False)
print("\nDistrict Average Temperatures (°C):")
print("==================================")
for _, row in result.iterrows():
    print(f"{row['DISTRICT']:<20} {row['temperature']:>6.2f}°C")

NameError: name 'ds1' is not defined